<a href="https://colab.research.google.com/github/Cypher29/AI_repo/blob/main/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoModel

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import pipeline
#import anthropic

In [ ]:
hf_token = userdata.get('HUGGINGFACE_KEY')
login(hf_token, add_to_git_credential=True)

In [ ]:
config = AutoModel.from_pretrained('google-bert/bert-base-uncased')

In [ ]:
print(config)

In [ ]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
QWEN2_MODEL_NAME = "Qwen/Qwen2-7B-Instruct"
STARCODER2_MODEL_NAME = "bigcode/starcoder2-3b"

In [ ]:
text = "Hello, Explain about software development life cycle"

In [ ]:
#text1 = "Hello, Explain about software development life cycle"
#tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english', trust_remote_code=True)
tokens = tokenizer.encode(text)
print(tokens)
print(tokenizer.batch_decode(tokens));

In [ ]:
#text1 = "Hello, Explain about software development life cycle"
star_tokenizer = AutoTokenizer.from_pretrained(STARCODER2_MODEL_NAME, trust_remote_code=True)
star_tokens = star_tokenizer.encode(text)

In [ ]:
#text2 = "Hello, Explain about software development life cycle"
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME, trust_remote_code=True)
phi3_tokens = phi3_tokenizer.encode(text)



In [ ]:
#text3 = "Hello, Explain about software development life cycle"
qwen2_tokenizer = AutoTokenizer.from_pretrained(QWEN2_MODEL_NAME)
qwen2_tokens = qwen2_tokenizer.encode(text)

In [ ]:


print(star_tokens)
print(star_tokenizer.batch_decode(star_tokens));
print()
print(phi3_tokens)
print(phi3_tokenizer.batch_decode(phi3_tokens));
print()
print(qwen2_tokens)
print(qwen2_tokenizer.batch_decode(qwen2_tokens));

This is sentiment analysis model

In [ ]:
len(tokenizer)

In [ ]:
print(tokenizer)

In [ ]:
text = "Hello, Explain about software development life cycle"
tokens = tokenizer.encode(text)
vocabsize = tokenizer.vocab_size
print("Vocab size : " ,vocabsize)
tokens

In [ ]:
len(tokens)

In [ ]:
tokenizer.decode(tokens)

In [ ]:
tokenizer.all_special_tokens

In [ ]:
tokenizer.batch_decode(tokens)

In [ ]:
tokenizer.get_added_vocab()

In [ ]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME)

text = "I am excited to show Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
tokens = phi3_tokenizer.encode(text)
print(phi3_tokenizer.batch_decode(tokens))

In [ ]:
tokenizer.all_special_tokens
tokenizer.get_vocab()


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("My bike has trouble and I feel good as I don't need to got to market ", return_tensors="pt")
print(inputs)
len(inputs)
with torch.no_grad():
    logits = model(**inputs).logits
print(logits)
predicted_class_id = logits.argmax().item()
print(predicted_class_id)
model.config.id2label[predicted_class_id]

In [ ]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
system_message = "You are an assistant that is great at completing sentense. Always be accurate. If you don't know the answer, say so."
#user_prompt = "Plucking apples from apple tree in apestreat. How many p's are there in above sentense"
user_prompt = "food ball is a game"
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
#gpt-3.5-turbo
#gpt-4.1
completion = openai.chat.completions.create(model='gpt-4.1', messages=prompts, temperature=0.8)
print(completion.choices[0].message.content)

In [ ]:
#completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
Response = openai.responses.create(model='gpt-4.1',  tools=[{"type": "web_search_preview"}],input="What is today's date")
print(Response.output_text)
print(Response)

#print(completion.choices[0].message.content)

In [ ]:
import math
response = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts,
                                        max_tokens=3,   temperature=0.8,  logprobs=True,     seed=42,         top_logprobs=7,        stream=True)
predictions = []
for chunk in response:
            if chunk.choices[0].delta.content:
                token = chunk.choices[0].delta.content
                logprobs = chunk.choices[0].logprobs.content[0].top_logprobs
                logprob_dict = {item.token: item.logprob for item in logprobs}

                # Get top predicted token and probability
                top_token = token
                top_prob = logprob_dict[token]

                # Get alternative predictions
                alternatives = []
                for alt_token, alt_prob in logprob_dict.items():
                    if alt_token != token:
                        alternatives.append((alt_token, math.exp(alt_prob)))
                alternatives.sort(key=lambda x: x[1], reverse=True)

                prediction = {'token': top_token, 'probability': math.exp(top_prob),'alternatives': alternatives[:3]}
                predictions.append(prediction)

print(predictions)

Quantization

In [ ]:
#!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate
!pip install -q requests  bitsandbytes sentencepiece accelerate

In [ ]:
from transformers import  AutoTokenizer,AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
#import gc

In [ ]:
#LLAMA = "meta-llama/Llama-4-Maverick-17B-128E-Instruct"
LLAMA = "bigcode/starcoder2-3b"

In [ ]:
messages = [
    {"role": "system", "content": "You are a good helpful assistant"},
    {"role": "user", "content": "Tell a proverb"}
  ]

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

In [ ]:
total_model_mem = model.get_memory_footprint() / 1e6
print(f"Memory : {total_model_mem:,.1f} MB")